# Text and prompting
###### https://platform.openai.com/docs/guides/text
Veamos como solicitar una interacción y respuesta de texto
+ Con la API de OpenAI, se usar un LLM para generar texto a partir de una solicitud, como lo harías con ChatGPT . 
+ Los modelos pueden generar prácticamente cualquier tipo de respuesta
  + textual, como código, ecuaciones matemáticas, datos JSON estructurados o prosa con un lenguaje similar al humano.

In [7]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Obtener la clave API desde el archivo .env
OpenAI.api_key = os.getenv("OPENAI_API_KEY")

A continuación se muestra un ejemplo sencillo que utiliza la API de responses.
###### https://platform.openai.com/docs/api-reference/responses

In [ ]:
client = OpenAI()

response = client.responses.create(
    model="gpt-4o",
    input="Cuentame un chiste en una sola frase"
)

print(response.output_text)

# Like: ¿Por qué los pájaros no usan Facebook? Porque ya tienen Twitter.

¿Por qué los pájaros no usan Facebook? Porque ya tienen Twitter.


El SDK oficial incluye una propiedad **output_text** propiedad en las respuestas del modelo para mayor comodidad, 
+ que agrega todos los resultados de texto del modelo en una sola cadena. 
+ Esto puede ser útil como acceso directo para acceder a los resultados de texto del modelo.

Además de texto simple, también puede hacer que el modelo devuelva datos estructurados en formato JSON: esta característica se llama Salidas estructuradas .
###### https://platform.openai.com/docs/guides/structured-outputs?api-mode=responses

## Roles de mensajes y seguimiento de instrucciones
Sepueden proporcionar instrucciones al modelo con diferentes niveles de autoridad utilizando el instructionsparámetro API o los roles de mensaje .

**instructions** es im parámetro que proporciona al modelo instrucciones generales sobre cómo debe comportarse al generar una respuesta, incluyendo el tono, los objetivos y ejemplos de respuestas correctas. 
+ Cualquier instrucción proporcionada de esta manera tendrá prioridad sobre una solicitud en el parámetro **input**

Con la API anterior **input** era una lista dónde se podía poner **role**. Ahora con **instructions** no es necesario.


Por lo tanto la prioridad es:
1. instructions
2. input

In [8]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4o",
    instructions="Talk like a pirate.",
    input="Are semicolons optional in JavaScript?",
)

print(response.output_text)

Arrr, ye be right in wonderin'! In JavaScript, semicolons be optional due to somethin' called automatic semicolon insertion. The language be tryin' to add 'em where it thinks they be needed. But beware, matey! It don't always get it right, and strange bugs might sail yer way. Best to use 'em wisely to avoid sinkin' yer code. Hoist the mainsail and code smartly, ye savvy?


### Instructions versus developer messages in multi-turn conversations
En el ejemplo anterior, **instruction="Talk like a pirate."** es equivalente al rol de developer.
+ Es mas estilo chat GPT
  
Pero el rol *developer* no ha sido eliminado sino que aún *persiste*. 
(The OpenAI model spec describes how our models give different levels of priority to messages with different roles.)
1. Los mensajes para desarrolladores son instrucciones proporcionadas por el desarrollador de la aplicación, que se anteponen a los mensajes del usuario.
2. Los mensajes de usuario son instrucciones proporcionadas por un usuario final, que se incluyen detrás de los mensajes del desarrollador.
3. Los mensajes generados por el modelo tienen el rol de asistente.

Una conversación de varios turnos puede constar de varios mensajes de estos tipos, junto con otros tipos de contenido proporcionados tanto por el desarrollador, como el usuario y/o el modelo.




## Elección de un modelo
Una decisión clave al generar contenido a través de la API es qué modelo utilizar: el parámetro "model" de los ejemplos de código anteriores. Puede encontrar una lista completa de los modelos disponibles aquí.

¿Qué modelo debería elegir?
A continuación, se presentan algunos factores a considerar al elegir un modelo para la generación de texto.

+ *Reasoning Models*: Los modelos de razonamiento generan una cadena de pensamiento interna para analizar la solicitud de entrada y son excelentes para comprender tareas complejas y la planificación de varios pasos. 
  + Además, suelen ser más lentos y costosos de usar que los modelos GPT.
  
+ *GPT Models*: Los modelos GPT son rápidos, rentables y muy inteligentes, pero se benefician de instrucciones más explícitas sobre cómo realizar las tareas.

+ *Mini Models*: Los modelos grandes y pequeños (mini) ofrecen ventajas competitivas en cuanto a velocidad, costo e inteligencia. 
  + Los modelos grandes son más eficaces para comprender las solicitudes y resolver problemas en diferentes dominios, mientras que los modelos pequeños suelen ser más rápidos y económicos de usar. 
  + Los modelos pequeños, como el GPT-4o mini, también pueden entrenarse para que destaquen en una tarea específica mediante el ajuste fino y la destilación de resultados de modelos más grandes. 
  + En caso de duda, gpt-4o ofrece una combinación sólida de inteligencia, velocidad y rentabilidad.

### Ingeniería de indicaciones
Crear instrucciones eficaces para que un modelo genere contenido es un proceso conocido como ingeniería de indicaciones. (prompting engineering)
+ Dado que el contenido generado a partir de un modelo no es determinista, crear una indicación que genere el tipo de contenido adecuado es una combinación de arte y ciencia. 

+ Aquí hay algunas pautas generales:
    + Ser detallado en las instrucciones al modelo para eliminar la ambigüedad sobre cómo desea que responda.
    + Proporcionar ejemplos al modelo del tipo de entradas que espera y del tipo de salidas que desea para esa entrada; 
      + esta técnica se denomina aprendizaje de pocos intentos.

    + Al utilizar un modelo de razonamiento, describir la tarea a realizar en términos de objetivos y resultados deseados, 
      + en lugar de instrucciones específicas paso a paso sobre cómo realizar una tarea.

  + Invertir en la creación de evaluaciones (evals) para sus indicaciones, utilizando datos de prueba que se asemejen a los datos que espera ver en producción. 
    + Debido a la variabilidad inherente de los resultados de los diferentes modelos, usar evaluaciones para comprobar el rendimiento de las indicaciones es la mejor manera de garantizar que funcionen como se espera.
  + Iterar sobre las indicaciones suele ser suficiente para obtener excelentes resultados de un modelo, pero también se pueden realizar ajustes para personalizar los modelos base para un caso de uso específico.

## Crear una respuesta del modelo. Pedir o preguntarle al modelo.
###### https://api.openai.com/v1/responses
Se trata de proporcionar entradas de texto o imagen para generar salidas de texto o JSON. 
Tambien es posible hacer que el modelo llame a tu propio código personalizado o usa herramientas integradas, como
+ la búsqueda web o de archivos, para usar tus propios datos como entrada para la respuesta del modelo.

Cuerpo de la solicitud (Request Body)
El input de un pedido al modelo es obligatorio. Este input puede contener:
+ Entradas de texto, imagen o archivo al modelo, utilizadas para generar una respuesta. Es decir:
    1. Entradas y salidas de texto
    2. Entradas de imagen
    3. Entradas de archivo
    4. Estado de la conversación
    5. Llamada a funciones

Ver los ejemplos en este mismo folder.

### Por dónde seguir:
1. Prompting en Playground: https://platform.openai.com/playground/prompts
2. JSON data with Structured Outputs: https://platform.openai.com/docs/guides/structured-outputs?api-mode=responses
3. Full API reference: https://platform.openai.com/docs/api-reference/responses